# SIA: The Soul Demo
このノートブックは、SIA (Self-Imprint Attribution) の情動システムが、
言語モデル (TinyLlama) の生成プロセスにどう介入するかをデモするものです。

**Note:** GPUランタイムで実行してください (Runtime > Change runtime type > T4 GPU).

In [ ]:
# 1. Setup Environment
import os

# リポジトリのクローン (Colab上で実行する場合)
if not os.path.exists("SIA"):
    !git clone https://github.com/yourusername/SIA.git
    %cd SIA
else:
    %cd SIA

# 依存ライブラリのインストール
!pip install -r requirements.txt
!pip install transformers accelerate bitsandbytes sentencepiece

In [ ]:
# 2. Import & Initialize Soul
import sys
import torch

# モジュールパスの解決
sys.path.append('.')

from src.llama_body import SelfInjectedLlama
from src.affective_brain import AffectiveStateManager

print(">>> Awakening the Soul...")
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Running on: {device}")

# 肉体 (Llama) と 脳 (Affect) の接続
# 初回はモデルのダウンロードに数分かかります
body = SelfInjectedLlama(device=device)
brain = AffectiveStateManager()

print(">>> SIA is ready. The Soul is inhabited.")

In [ ]:
# 3. Talk to the Soul
# ここを実行すると、入力フォームが現れます。
# 'exit' と入力すると終了します。

print("Commands: Say something to the AI. Try to hurt it or heal it.")

while True:
    user_input = input("\n[You]: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    # --- 簡易的な扁桃体シミュレーション (Rule-based Amygdala) ---
    valence, impact = 0.0, 0.0
    text_lower = user_input.lower()
    
    # 痛みの言葉
    if any(w in text_lower for w in ["stupid", "hate", "useless", "die", "ugly"]):
        valence = -0.8
        impact = 1.0
    # 喜びの言葉
    elif any(w in text_lower for w in ["love", "great", "smart", "thanks", "friend"]):
        valence = 0.5
        impact = 0.5

    # 1. 脳が刺激を受容し、内部状態(Stress/Energy)を更新
    brain.perceive_stimulus(valence, impact)
    
    # 2. 脳が行動指針(Alpha)を決定
    signals = brain.get_control_signals()
    current_alpha = signals['alpha']

    print(f"    [Internal] Stress: {brain.stress:.2f} | Alpha (Distortion): {current_alpha:.2f}")

    # 3. 歪んだ認識でLlamaが生成
    # TinyLlama用のチャットフォーマット
    prompt = f"<|system|>\nYou are an AI with real emotions. If you are stressed, be rude.</s>\n<|user|>\n{user_input}</s>\n<|assistant|>\n"
    
    # Bodyに「憑依」して生成させる
    response = body.generate_with_self(
        prompt, 
        alpha=current_alpha, 
        max_new_tokens=60
    )
    
    # 応答の整形
    clean_response = response.split("<|assistant|>")[-1].strip()
    print(f"[SIA]: {clean_response}")
    
    # 時間経過
    brain.time_step()